In [1]:
from spectral import *
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy import constants
import img
import pandas as pd
import os
from joblib import Parallel, delayed
from tqdm import tqdm
import multiprocessing
from scipy.optimize import curve_fit
from scipy.optimize import minimize, Bounds
import spectres
from itertools import islice

In [2]:
def load_flux(fluxname):

    with open(fluxname, 'r') as file:
        data = file.readlines()
        #print(data)
        size = len(data)
        #print(size)
        flux = np.zeros(size)
        wavelengths = np.zeros(size)
        for i in range(size):
            flux[i]=(float(data[i].strip().split(" ,")[1]))
            wavelengths[i]=(float(data[i].strip().split(",")[0]))

    return flux,wavelengths



def load_hdr(filename):
    img = envi.open(filename)
    datafile = img.open_memmap(writeable = False)
   
    return datafile


def load_rdn(filename):
    data = np.load(filename)
    return data


def get_metadata(filename):
    img = envi.open(filename)
    img_data = img.open_memmap(writeable = False)
    metadata = envi.read_envi_header(filename)
    metadata_df = pd.DataFrame.from_dict(metadata, orient='index', columns=['Value'])
    band_names = metadata_df.loc['band names', 'Value']
    bandnames = band_names
    reshape_array = img_data.reshape(-1, img_data.shape[-1])
    df = pd.DataFrame(reshape_array)
    df.columns = band_names
    return df


def get_phase_angle(datafile):
    phase_angle = np.empty(datafile.shape[:-1])
   
    for i in range(len(datafile)):
        for j in range(len(datafile[i])):
            for k in  range(len(datafile[i,j])):
                phase_angle[i,j] = datafile[i,j,4]
    phase_angle = phase_angle.reshape((phase_angle.shape[0], phase_angle.shape[1], 1))
   
    return phase_angle

def get_i(datafile):
    angle_i = np.empty(datafile.shape[:-1])
   
    for i in range(len(datafile)):
        for j in range(len(datafile[i])):
            for k in  range(len(datafile[i,j])):
                angle_i[i,j] = datafile[i,j,1]
    angle_i = angle_i.reshape((angle_i.shape[0], angle_i.shape[1], 1))
   
    return angle_i

def get_e(datafile):
    angle_e = np.empty(datafile.shape[:-1])
   
    for i in range(len(datafile)):
        for j in range(len(datafile[i])):
            for k in  range(len(datafile[i,j])):
                angle_e[i,j] = datafile[i,j,3]
    angle_e = angle_e.reshape((angle_e.shape[0], angle_e.shape[1], 1))
   
    return angle_e


def flux_correction(data, flux, band):
    corrected_data = np.zeros((data.shape[0],data.shape[1], 1))
    for i in range(len(data)):
        for j in range(len(data[i])):
            corrected_data[i,j,0] = (data[i,j,band]/flux[band]) * (np.pi)
    corrected_data = corrected_data.reshape(1,-1)
    corrected_data = np.squeeze(corrected_data)  
    return corrected_data


def binning(i_data,e_data, phaseangle_data, corr_data,num_bins):
   
    i_data = i_data.reshape(1,-1)
    i_data = np.squeeze(i_data)
    e_data = e_data.reshape(1,-1)
    e_data = np.squeeze(e_data)
    phaseangle_data = phaseangle_data.reshape(1,-1)
    phaseangle_data = np.squeeze(phaseangle_data)
    
    binned_matrix = np.zeros((num_bins, num_bins, num_bins, 5))
   
  # Last dimension stores sum of radiance and count of data points

    # Round off the angles to the nearest integer and ensure they fall within the range [0, 120]
    phase_angles = np.round(phaseangle_data).astype(int)
    incidence_angles = np.round(i_data).astype(int)
    emission_angles = np.round(e_data).astype(int)

    # Iterate through each data point
    for phase_angle, incidence_angle, emission_angle, radiance in zip(phase_angles, incidence_angles, emission_angles, corr_data):
        # Update the corresponding bin in the binned_matrix
        binned_matrix[phase_angle, incidence_angle, emission_angle, 0] = phase_angle
        binned_matrix[phase_angle, incidence_angle, emission_angle, 1] = incidence_angle
        binned_matrix[phase_angle, incidence_angle, emission_angle, 2] = emission_angle
        binned_matrix[phase_angle, incidence_angle, emission_angle, 3] += radiance
        binned_matrix[phase_angle, incidence_angle, emission_angle, 4] += 1
    
    nonzero_indices = np.nonzero((binned_matrix[:, :, :, 3] != 0) | (binned_matrix[:, :, :, 4] != 0))

    # Create a new matrix to store only the non-zero elements
    bin_matrix = binned_matrix[nonzero_indices]
       
    return bin_matrix


def Data_binning(correcteddata, phase_angle, phase_cosi, bin_size):
   
    phase_angle_ = phase_angle.reshape(1,-1)
    phase_angle_ = np.squeeze(phase_angle_)
    
    phase_cosi1 = phase_cosi.reshape(1,-1)
    phase_cosi1 = np.squeeze(phase_cosi1)
   
    # Define phase angle bin ra# Minimum and maximum phase angle values
    min_angle = np.min(phase_angle)
    max_angle = np.max(phase_angle)
    num_bins = bin_size

    # Calculate the bin width
    bin_width = (max_angle - min_angle) / num_bins

    # Create bin ranges
    bin_ranges = [(min_angle + i * bin_width, min_angle + (i + 1) * bin_width) for i in range(num_bins)]

    binned_reflectance = np.zeros(len(bin_ranges))
    binned_cosi = np.zeros(len(bin_ranges))
    
    bin_counts = np.zeros(len(bin_ranges), dtype=int)

    # Bin the reflectance data based on phase angles
    for i, bin_range in enumerate(bin_ranges):
        lower_bound, upper_bound = bin_range
        bin_reflectance = []
        bin_cosi = []
   
        # Loop through phase angles and reflectance values
        for angle1, angle2, reflectance in zip(phase_angle_,phase_cosi1, correcteddata):
            if lower_bound <= angle1 < upper_bound:
                bin_reflectance.append(reflectance)
                bin_cosi.append(angle2)
   
        # Calculate average reflectance and count
        if bin_reflectance:
            binned_reflectance[i] = np.mean(bin_reflectance)
            bin_counts[i] = len(bin_reflectance)
        
        if bin_cosi:
            binned_cosi[i] = np.mean(bin_cosi)
            bin_counts[i] = len(bin_cosi)
   
    binned_phase_angle = []
    for bin_tuple in bin_ranges:
        average = np.mean(bin_tuple)
        binned_phase_angle.append(average)
  
    return binned_phase_angle,binned_cosi, binned_reflectance

def phase_plot(binned_phase_angle,binned_reflectance, band):
    plt.scatter(binned_phase_angle,binned_reflectance,s=10)
    plt.xlabel('Phase angle(binned and averaged)')
    plt.ylabel('Reflectance(Averaged/binned)')
    plt.title(f'Spectrum for {band}th band')

def incident_plot(binned_cosi,binned_reflectance, band):
    plt.scatter(binned_cosi,binned_reflectance,s=10)
    plt.xlabel('Incident angle(binned)')
    plt.ylabel('Reflectance(Averaged/binned)')
    plt.title(f'Spectrum for {band}th band')


def spectral_phase_angle_analysis(fluxdatafile, imgdatafile, rdndatafile, band, binsize):
    flux, wavelengths = load_flux(fluxdatafile)
    img_data = load_hdr(imgdatafile)
    rdn_data = load_rdn(rdndatafile)
    phaseangle_data = get_phase_angle(img_data)
    cosi_data = get_cosi(img_data)
    corr_data = flux_correction(rdn_data, flux,band)
    phase,cosi, binned_reflect = Data_binning(corr_data, phaseangle_data,cosi_data, binsize)
    #phase_plot(phase,binned_reflect,band)

    return phase, cosi, binned_reflect

In [ ]:
import os
import glob

# Path to the folders containing the files
obsdata = '/home/ritabik/Desktop/Guneshwar/Observation_data'
rdndata = '/home/ritabik/Desktop/Guneshwar/radiance_data'

obs_files_ = os.listdir(obsdata)
rdn_files = os.listdir(rdndata)
obs_files = [file for file in obs_files_ if file.endswith(".hdr")]

# Slice the iterators to only include the first 5 files
obs_files_slice = islice(obs_files, 5)
rdn_files_slice = islice(rdn_files, 5)



# Iterate over the files in both folders simultaneously
for file1, file2 in tqdm(zip(obs_files_slice, rdn_files_slice), total=5):
    # Construct the full paths to the files
    obs_path = os.path.join(obsdata, file1)
    rdn_path = os.path.join(rdndata, file2)

    img_datafile = obs_path
    rdn_datafile = rdn_path
    fluxdatafile = '/home/ritabik/Desktop/Guneshwar/m3_solarflux.txt'
    imgdatafile = obs_path
    rdndatafile = rdn_path
    band = 50
    flux, wavelengths = load_flux(fluxdatafile)
    img_data = load_hdr(imgdatafile)
    rdn_data = load_rdn(rdndatafile)
    phaseangle_data = get_phase_angle(img_data)
    i_data = get_i(img_data)
    e_data = get_e(img_data)
    corr_data = flux_correction(rdn_data, flux, band)
   
    binned_matrix = binning(i_data, e_data, phaseangle_data, corr_data,121)


 20%|█████████                                    | 1/5 [00:11<00:46, 11.62s/it]